# Infrastructure Inequality Analysis (Los Angeles)

This notebook documents the analytical workflow and key findings of a Snowflake-based analysis on construction permit patterns across income groups in Los Angeles.

> Note: Due to environment constraints, Snowflake execution and visual outputs are generated externally. Figures are embedded for presentation and interpretation purposes.

# Import Python Libraries

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st

# Initialize the Active Snowflake Session
### What this does
This code retrieves the currently active Snowflake session within the Snowflake Python/Snowpark environment.
	•	The session object represents an authenticated connection to Snowflake
	•	It allows the notebook to execute SQL commands and access Snowflake tables directly from Python

In short, this step establishes the bridge between Snowflake and Python.

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session() 

# Set the Snowflake Execution Context (Role, Database, Schema)
### What this does

These commands define the working context inside Snowflake:
	•	Role (TRAINING_ROLE)
Controls access permissions to databases and tables
	•	Database (LA_PERMIT_DATA)
Specifies the database containing Los Angeles permit data
	•	Schema (PUBLIC)
Defines the namespace for tables used in the analysis

The .collect() call forces execution of each SQL statement, as Snowpark uses lazy evaluation by default.

In [ ]:
session.sql("USE ROLE TRAINING_ROLE").collect() 
session.sql("USE DATABASE LA_PERMIT_DATA").collect()
session.sql("USE SCHEMA PUBLIC").collect()

# Load Snowflake Tables into Pandas DataFrames
### What this does

This section extracts multiple Snowflake tables and converts them into Pandas DataFrames for analysis.

Data flow
	1.	session.table("TABLE_NAME") references a Snowflake table
	2.	.to_pandas() converts the table into a Pandas DataFrame
	3.	The resulting DataFrames are used for downstream analysis, visualization, and dashboard creation

Table overview
	•	PERMIT_RECORDS – Core building permit records
	•	CENSUS_TRACTS – Demographic and geographic census data
	•	MASTER_LICENSE – Contractor license information
	•	PERSONNEL_DATA – Personnel-related records
	•	WORKER_COMP – Worker compensation and insurance data

In [ ]:
records        = session.table("PERMIT_RECORDS").to_pandas()
census_tracts  = session.table("CENSUS_TRACTS").to_pandas()
contractors    = session.table("MASTER_LICENSE").to_pandas()
personnel      = session.table("PERSONNEL_DATA").to_pandas()
worker_comp    = session.table("WORKER_COMP").to_pandas()

# Project Goal: 
### Understanding whether there is a difference between low income's and high income’s Building Permit type?
-- Low income: AMI_CAtegopry include "low"; High income: AMI_Category is "Above Moderate Income".

### First Step: Clean and Combine PERMIT_RECORDS with CENSUS_TRACTS data

In [ ]:
records['CT_To_Combine'] = pd.to_numeric(records['CENSUS_TRACT'], errors='coerce')*100+6037000000
records_with_census = records.merge(census_tracts, left_on='CT_To_Combine', right_on='CENSUS_TRACT', how='inner')
records_with_census = records_with_census[['AMI_CATEGORY', 'PERMIT_TYPE']].dropna()

# Start Our Analysis
## Part 1: High Income and Low income has different Construction Priorities
To begin understanding how income inequality influences the way communities manage their
built environments, this first figure compares the permit type composition of low-income and
high-income neighborhoods. By looking at the share of permit types within each income group,
we can observe differences in what kinds of construction activities each community most
frequently undertakes.

In [ ]:
low_income = records_with_census[records_with_census['AMI_CATEGORY'].str.contains(r'Low')]
high_income = records_with_census[records_with_census['AMI_CATEGORY'].str.contains(r'Above Moderate Income')]

low_counts = low_income['PERMIT_TYPE'].value_counts(normalize=True)   # 加 normalize=True 得比例
high_counts = high_income['PERMIT_TYPE'].value_counts(normalize=True)

combined_share = pd.DataFrame({
    'Low Income Share': low_counts,
    'High Income Share': high_counts
})

combined_share.plot(kind='bar', figsize=(14,6))
plt.xlabel('Permit Type')
plt.ylabel('Share of Permits')
plt.title('Permit Type Share Comparison Across Income Groups')
plt.savefig(
    "../assets/figures/permit_type_share_by_income.png",
    bbox_inches="tight",
    dpi=150
)
plt.show()

### Permit Type Distribution by Income Group

![Permit Type Share Comparison](../assets/figures/permit_type_share_by_income.png)

#### Insight:
Low-income areas show a strong concentration in essential repair and maintenance permits,
especially Alter/Repair, Electrical, and Fire Sprinkler. This suggests that much of the
construction activity in these neighborhoods is reactive: addressing aging infrastructure, fixing
basic systems, and keeping existing buildings functional.
In contrast, high-income communities allocate a larger share of permits toward
value-enhancing improvements, such as swimming pools, grading, new non-building
structures, and various site upgrades. These categories reflect proactive investment in
upgrades that improve comfort, aesthetics, or long-term property value.
This initial comparison establishes an important baseline:
Different income groups are engaging in fundamentally different types of construction.
Low-income neighborhoods spend more of their effort maintaining what already exists, while
high-income neighborhoods invest more broadly in improvements and amenities.

## Part 2: Essential vs. Non-Essential Construction for Low and High Income groups
The second figure compares how much of each income group’s construction activity falls into
Essential Construction, which includes repairs, utility work, demolition, fire safety systems,
and other categories necessary for maintaining basic building functionality. All remaining permit
types are grouped into Other Improvements, representing upgrades, additions, and
value-enhancing projects.
Code: (I don’t know how to create pie chart in one figure, so I ask ChatGpt to help me
with the part below #plotting. I ask: Can you help me create only one plot based on high
income and low income’s respective matrix)

In [ ]:
# Define which are essential constructions
essential_categories = [
    'Bldg-Alter/Repair',
    'Nonbldg-Alter/Repair',
    'Bldg-Demolition',
    'Electrical',
    'Plumbing',
    'Fire Sprinkler',
    'Bldg-Relocation'
]

# Low income share
low_all = combined_share['Low Income Share']
# Essential 佔比
low_essential = low_all[low_all.index.isin(essential_categories)].sum()
# Others（非 essential 的全部加起來）
low_others = 1 - low_essential
# low income's Pie's number
low_pie = pd.Series([low_essential, low_others], index=['Essential Construction', 'Others'])

# High income shares
high_all = combined_share['High Income Share']
# Essential 佔比
high_essential = high_all[high_all.index.isin(essential_categories)].sum()
# Others
high_others = 1 - high_essential
# low income's Pie's number
high_pie = pd.Series([high_essential, high_others], index=['Essential Construction', 'Others'])

# Plotting
plt.figure(figsize=(14, 6))
# Low Income Pie
plt.subplot(1, 2, 1)
plt.pie(low_pie, labels=low_pie.index, autopct='%1.1f%%')
plt.title('Low Income: Essential vs Others')
# High Income Pie
plt.subplot(1, 2, 2)
plt.pie(high_pie, labels=high_pie.index, autopct='%1.1f%%')
plt.title('High Income: Essential vs Others')

plt.show()

### Essential vs Non-essential permit Distribution by Income Group

![essential vs non-essential permit Distribution Comparison](../assets/figures/Essential_vs_Nonessential_by_income.png)

#### Insight:
The contrast between the two pie charts is striking!!!!!
Low-income neighborhoods allocate the majority of their permit activity to Essential
Construction. This means that most of their construction work is tied to keeping buildings safe,
operational, and up to minimum standards, such as repairing deteriorating structures, fixing
electrical systems, addressing plumbing issues, and maintaining fire sprinkler systems. The
large Essential slice indicates that these communities are primarily responding to immediate
needs and structural deficiencies.
In contrast, high-income neighborhoods show a much smaller share of permits in the
Essential category, with a significantly larger portion falling under Other Improvements. This
suggests that residents in high-income areas spend less on necessary repairs and more on
lifestyle or property-enhancing upgrades, such as pools, grading work, site improvements, and
non-essential additions. The broader distribution in the high-income pie highlights a greater
capacity to invest in discretionary or value-adding projects rather than urgent maintenance.
Overall, these two pie charts provide a clear visual confirmation of the pattern identified earlier:
low-income communities focus their construction activity on maintaining what already
exists, whereas high-income communities invest more heavily in improving and
enhancing their properties.

## Part 3: Which Permit Types Show the Largest Income Gap?
To pinpoint the specific activities where income differences are most pronounced, the third figure
plots the ratio of high-income to low-income permit shares for every permit category.
A ratio above 1 indicates the permit type is more common in high-income neighborhoods, while
a ratio below 1 indicates the opposite.

In [ ]:
# 取得高收入與低收入各 permit 的比例
low = combined_share['Low Income Share']
high = combined_share['High Income Share']
# 計算 ratio
ratio = high / low
# Plot
ratio.sort_values().plot(kind='barh')
plt.xlabel('High Income Share / Low Income Share')
plt.title('Permit Type Ratio: High Income vs Low Income')
plt.show()

### Permit Type ratio: High vs Low Income

![Permit Type ratio Comparison](../assets/figures/permit_type_ratio.png)

#### Insight:
The results reveal several strong contrasts:
High-income neighborhoods are far more likely to request discretionary or amenity-focused
permits, such as Swimming Pool/Spa, Grading, New Non-building Construction, and various site
improvement projects. These categories show ratios many times higher in high-income
communities, highlighting their greater capacity to invest in comfort, aesthetics, and long-term
property value.
Low-income neighborhoods disproportionately request essential safety and maintenance
permits, including Alter/Repair, Electrical, Plumbing, and Fire Sprinkler work. The ratio for these
categories is well below 1, indicating a much heavier reliance on repairs and basic functionality
upgrades.
This ratio-based comparison makes the income divide especially clear: high-income areas
specialize in upgrades, while low-income areas specialize in survival-level maintenance.

## Part 4: Construction Need Concentration
This figure shows the cumulative share of permit activity after reordering categories so that all
essential construction types appear first, followed by all non-essential types.
This allows us to visualize how quickly each income group accumulates construction activity
within the essential category.
Code:
I asked the AI how to plot a cumulative curve using only the data frame I already created. And it
provided me with the code below the # Cumulated Sum. Then I remove code I think is useless, such
as plt.legend(), plt.grid(True), plt.tight_layout().

In [ ]:
essential = [
    'Bldg-Alter/Repair',
    'Nonbldg-Alter/Repair',
    'Bldg-Demolition',
    'Electrical',
    'Plumbing',
    'Fire Sprinkler',
    'Bldg-Relocation'
]

# 原始比例
low = combined_share['Low Income Share']
high = combined_share['High Income Share']

# 排序：先 essential，再 non-essential
low_ordered = pd.concat([low[low.index.isin(essential)], low[~low.index.isin(essential)]])
high_ordered = pd.concat( [ high[high.index.isin(essential)], high[~high.index.isin(essential)]] )

# Cumulated Sum
low_cum = low_ordered.cumsum()
high_cum = high_ordered.cumsum()

plt.figure(figsize=(10,6))
plt.plot(low_cum.values, label='Low Income')
plt.plot(high_cum.values, label='High Income')
plt.xticks(range(len(low_ordered)), low_ordered.index, rotation=45, ha='right')
plt.xlabel('Permit Types (Essential → Non-Essential)')
plt.ylabel('Cumulative Share')
plt.title('Cumulative Share by Functional Order')
plt.legend()
plt.show()

### Cumulative Share by Functional Order

![Cumulative Share by Functional Order](../assets/figures/cumulative_share_by_functional_order.png)

#### Insight:
The results reveal strong contrasts in both groups’ accumulating speed:
Low-income neighborhoods show a steep rise during the essential segment, meaning that a
large portion of their construction activity is absorbed by the seven essential types alone. This
indicates strong dependence on repairs, utility fixes, and safety-related work.
High-income neighborhoods accumulate more slowly in the essential segment, showing a flatter
curve. Their cumulative share rises more sharply only once non-essential improvements are
included, reflecting greater engagement in upgrades and value-adding projects.
Reordering the categories in this functional sequence makes the contrast even clearer:
low-income communities concentrate most of their construction needs in essential
maintenance, while high-income communities distribute their activity more heavily into
non-essential improvements.

## Part 5: Direct Differences in Permit Type Shares
To summarize how income shapes construction behavior, this final figure plots the direct
difference between high-income and low-income permit shares for every category.

In [ ]:
# 高收入 share - 低收入 share
difference = combined_share['High Income Share'] - combined_share['Low Income Share']

# 排序（最清楚）
difference_sorted = difference.sort_values()

plt.figure(figsize=(10,8))
difference_sorted.plot(kind='barh')
plt.xlabel('Difference (High Income Share - Low Income Share)')
plt.title('Permit Type Differences Between Income Groups')
plt.show()

### Permit Type Differences between Income Groups

![Permit Type Differences between Income Groups](../assets/figures/permit_type_differences.png)

#### Insight:
Positive values indicate permit types more common in high-income neighborhoods, while
negative values represent categories more prevalent in low-income neighborhoods.
The results provide a concise and powerful overview:
High-income neighborhoods show strong positive differences in amenity and
improvement-focused categories, such as Swimming Pool/Spa, Grading, New Non-Building
Construction…upgrades. These categories appear far more frequently in higher-income areas,
reflecting discretionary investment capacity.
While low-income neighborhoods show negative differences in essential repair and
safety-related categories, including Alter/Repair, Electrical, Plumbing, and Fire Sprinkler. These
permit types are more frequently requested in lower-income areas, consistent with a greater
need for basic maintenance and building functionality preservation.
By visualizing differences directly, this final figure reinforces the central finding of the analysis:
Income level strongly influences the types of construction activities pursued. High-income
communities invest in improvements, while low-income communities focus on essential repairs.